<a href="https://colab.research.google.com/github/Livia-dias/DataScience/blob/master/Copy_of_McDs_livia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy

In [ ]:
input_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Indigenous\indigenous_mapbiomas.shp"
output_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Indigenous\indigenous_mapbiomas_alb.shp"

In [ ]:
albers_wkt = """
PROJCS["Conica_Equivalente_de_Albers_Brasil",
GEOGCS["GCS_SIRGAS2000",
DATUM["D_SIRGAS2000",
SPHEROID["Geodetic_Reference_System_of_1980",6378137,298.2572221009113]],
PRIMEM["Greenwich",0],
UNIT["Degree",0.017453292519943295]],
PROJECTION["Albers"],
PARAMETER["standard_parallel_1",-2],
PARAMETER["standard_parallel_2",-22],
PARAMETER["latitude_of_origin",-12],
PARAMETER["central_meridian",-54],
PARAMETER["false_easting",5000000],
PARAMETER["false_northing",10000000],
UNIT["Meter",1]]
"""


In [ ]:
spatial_ref = arcpy.SpatialReference()
spatial_ref.loadFromString(albers_wkt)

arcpy.management.Project(
    in_dataset=input_shapefile,
    out_dataset=output_shapefile,
    out_coor_system=spatial_ref
)

print("Shapefile reprojetado com sucesso para Albers SIRGAS 2000!")

In [ ]:
import geopandas as gpd
import pandas as pd

# Caminho do arquivo shapefile
input_file = r"C:\arcgis\prov_dc_dissolve.shp"
output_file = r"C:\arcgis\prov_dc_dissol_area.shp"

# Forçar leitura com codificação latin1
gdf = gpd.read_file(input_file, encoding='latin1')

# Calcular a soma total da coluna SUM_Area_k
total_area_value = gdf['SUM_Area_k'].sum()

# Criar a nova coluna Total_area com o mesmo valor para todas as linhas
gdf['Total_area'] = total_area_value

# Exibir informações
print(f"Soma total de SUM_Area_k: {total_area_value}")
print(f"Projeção mantida: {gdf.crs}")
print(f"\nPrimeiras linhas do resultado:")
print(gdf[['SUM_Area_k', 'Total_area']].head())

# Salvar o arquivo mantendo a projeção original
gdf.to_file(output_file)

print(f"\nArquivo salvo como: {output_file}")
print(f"Total de linhas: {len(gdf)}")

In [ ]:
# Accu_sum
import arcpy

# Path to your feature class or shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Intersect\protected_areas\bolivia_PA_prov_Dissolve.shp"
# Update with your actual path

# Field name where you want to store the accumulated sum
accumulated_field = "Accu_sum"

# Field with the division result
division_field = "Area_prop"

# Sorting the table by the Area_prop field to ensure the accumulation happens in ascending order
# Create a sorted version of the shapefile based on Area_prop field
sorted_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Intersect\protected_areas\bo_prov_pa_dissolve_sorted_Area_prop.shp"
arcpy.management.Sort(fc, sorted_fc, [division_field])  # Sorting by the 'Area_prop' field

# Create a cursor to update the rows
total = 0  # This will hold the running total

# Use UpdateCursor to iterate through each row and accumulate the sum
with arcpy.da.UpdateCursor(sorted_fc, [division_field, accumulated_field]) as cursor:
    for row in cursor:
        # Add the division result to the cumulative total
        total += row[0]  # row[0] is the division result field (change if needed)
        row[1] = total  # Set the accumulated sum to the new field
        cursor.updateRow(row)

arcpy.AddMessage("Accumulated sum calculation completed!")


In [ ]:
## Neglig

import arcpy

# Path to your feature class or shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Intersect\prov_dc_dissolve_sorted_Area_prop.shp"

# Field names
accumulated_field = "Accu_sum"
category_field = "Neglig"  # New field to store the "Negligible" value

# Add a new field to store the category (if it doesn't already exist)
field_names = [f.name for f in arcpy.ListFields(fc)]
if category_field not in field_names:
    arcpy.AddField_management(fc, category_field, "TEXT", field_length=50)

# Use an UpdateCursor to iterate through each row and apply the logic
with arcpy.da.UpdateCursor(fc, [accumulated_field, category_field]) as cursor:
    for row in cursor:
        # Check if the Accumulated_Sum is less than or equal to 0.05
        if row[0] <= 0.050:
            row[1] = "Negligible"  # Set the value to "Negligible"
        else:
            row[1] = "To_Jenks"  # Leave empty if the condition is not met

        cursor.updateRow(row)  # Update the row with the new value

print("Categorization completed!")


In [ ]:
## Normalized

import arcpy

# Path to your feature class or shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Intersect\indigenous\bo_IL_prov_dissolve"

# Field names
area_field = "Area_prop"  # The field with the area values
normalized_field = "Normalized"  # The new field for normalized values

# Get the minimum and maximum values from the 'Area_prop' field
min_value = None
max_value = None

with arcpy.da.SearchCursor(fc, [area_field]) as cursor:
    for row in cursor:
        if min_value is None or row[0] < min_value:
            min_value = row[0]
        if max_value is None or row[0] > max_value:
            max_value = row[0]

# Print the min and max values for debugging purposes
print(f"Min Value: {min_value}")
print(f"Max Value: {max_value}")

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(fc)]
if normalized_field not in field_names:
    arcpy.AddField_management(fc, normalized_field, "DOUBLE")

# Normalize the 'Area_prop' values and populate the new field
with arcpy.da.UpdateCursor(fc, [area_field, normalized_field]) as cursor:
    for row in cursor:
        # Avoid division by zero in case all values are the same
        if max_value != min_value:
            # Apply normalization formula: (Value - Min) / (Max - Min)
            normalized_value = (row[0] - min_value) / (max_value - min_value)
        else:
            normalized_value = 0  # If all values are the same, set normalized value to 0

        # Update the normalized value in the new field
        row[1] = normalized_value
        cursor.updateRow(row)

print("Normalization completed!")

In [ ]:
## risk classification

import geopandas as gpd
import pandas as pd

# Caminho para o shapefile original
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Intersect\indigenous\bo_IL_prov_dissolve_tojenks"

# Carrega o shapefile
shapefile = gpd.read_file(shapefile_path)

# Verifica se a coluna 'IL_RISK' existe
if 'Normalized' in shapefile.columns:
    # Checa valores nulos
    print("\nChecking for NaN values in the 'Normalized' column:")
    print(shapefile['Normalized'].isna().sum())

    # Define os intervalos e rótulos
    bins = [0.062060, 0.198714 , 0.335794 , 0.511792, 1.000000]
    labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

    # Cria a coluna 'Jenks' com base em IL_RISK
    shapefile['Jenks'] = pd.cut(shapefile['Normalized'], bins=bins, labels=labels, right=True, include_lowest=True)

    # Converte para string por segurança
    shapefile['Jenks'] = shapefile['Jenks'].astype(str)

    # Salva por cima do shapefile original
    shapefile.to_file(shapefile_path)

    print("\nShapefile atualizado com sucesso com a nova coluna 'Jenks'.")
else:
    print("\nA coluna 'Normalized' não existe no shapefile. Verifique o nome.")



In [ ]:
#pastDC

import arcpy

# Caminho para o shapefile original
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Results\bo_pastdc_risk_jurisd.shp"

# Nome da nova coluna
new_field = "Jenks"

# Verifica se o campo já existe e o remove
fields = [f.name for f in arcpy.ListFields(shapefile_path)]
if new_field in fields:
    arcpy.DeleteField_management(shapefile_path, new_field)

# Adiciona o novo campo
arcpy.AddField_management(shapefile_path, new_field, "TEXT")

# Define os intervalos e rótulos
bins = [0.060638 , 0.108141, 0.197441 , 0.375992, 0.595708]
labels = ['Negligible','Very Low', 'Low', 'Medium', 'High', 'Very High']

# Atualiza os valores com cursor
with arcpy.da.UpdateCursor(shapefile_path, ['Normalized', new_field]) as cursor:
    for row in cursor:
        value = row[0]
        if value is None:
            row[1] = None
        elif value <= bins[0]:
            row[1] = labels[0]
        elif value <= bins[1]:
            row[1] = labels[1]
        elif value <= bins[2]:
            row[1] = labels[2]
        elif value < bins[3]:
            row[1] = labels[3]
        elif value < bins[4]:
            row[1] = labels[4]
        else:
            row[1] = labels[5]
        cursor.updateRow(row)

print("\nShapefile atualizado com sucesso com a nova coluna 'Jenks'.")

In [ ]:
## Create Unique ID

import subprocess
import os

# Caminho para a GDB e pasta de saída
gdb_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Jurisdiction\Geobolivia\SUNIT\limites_provinciales_alb.shp"
output_folder = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Jurisdiction\Geobolivia\SUNIT"

# Cria a pasta de saída se não existir
os.makedirs(output_folder, exist_ok=True)

# Lista as camadas da GDB
list_cmd = ["ogrinfo", gdb_path]
layers_info = subprocess.check_output(list_cmd, shell=True).decode("utf-8")
print(layers_info)

# Para cada camada, exporta para Shapefile
for line in layers_info.splitlines():
    if line.strip().startswith("1:") or line.strip().startswith("2:"):
        layer_name = line.split(":")[1].strip()
        output_shp = os.path.join(output_folder, f"{layer_name}.shp")
        cmd = ["ogr2ogr", "-f", "ESRI Shapefile", output_shp, gdb_path, layer_name]
        subprocess.run(cmd, shell=True)

In [ ]:
#protected areas

import arcpy

# Caminho do shapefile original
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Intersect\indigenous\bo_IL_prov_dissolve_tojenks.shp"

# Adiciona uma nova coluna "Jenks" se ela não existir
field_name = "Jenks"
if field_name not in [f.name for f in arcpy.ListFields(shapefile_path)]:
    arcpy.AddField_management(shapefile_path, field_name, "TEXT", field_length=20)

# Define os intervalos e rótulos (bins e labels)
bins = [0, 0.082112, 0.232342, 0.467742, 0.767778, 1.000000]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# Atualiza a coluna "Jenks" com base em "Normalized"
with arcpy.da.UpdateCursor(shapefile_path, ["Normalized", field_name]) as cursor:
    for row in cursor:
        value = row[0]
        if value is None:
            row[1] = None
        else:
            # Verifica em qual intervalo o valor se encaixa
            if value <= bins[0]:
                row[1] = labels[0]
            elif value <= bins[1]:
                row[1] = labels[1]
            elif value <= bins[2]:
                row[1] = labels[2]
            elif value <= bins[3]:
                row[1] = labels[3]
            else:
                row[1] = labels[4]
        cursor.updateRow(row)

print("✅ Shapefile atualizado com sucesso com a nova coluna 'Jenks'!")


Parte 2 Livia - Bolivia

In [ ]:
import arcpy

# Caminho do shapefile original
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Correction\Indigenous_Land\bo_IL_territorial_final.shp"

# Adiciona uma nova coluna "Jenks" se ela não existir
field_name = "Jenks"
if field_name not in [f.name for f in arcpy.ListFields(shapefile_path)]:
    arcpy.AddField_management(shapefile_path, field_name, "TEXT", field_length=20)

# Define os intervalos e rótulos (bins e labels)
bins = [0.082112, 0.232342, 0.467742, 0.767778]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# Atualiza a coluna "Jenks" com base em "Normalized"
with arcpy.da.UpdateCursor(shapefile_path, ["Normalized", field_name]) as cursor:
    for row in cursor:
        value = row[0]
        if value is None:
            row[1] = None
        else:
            # Verifica em qual intervalo o valor se encaixa
            if value < bins[0]:
                row[1] = labels[0]
            elif value < bins[1]:
                row[1] = labels[1]
            elif value < bins[2]:
                row[1] = labels[2]
            elif value < bins[3]:
                row[1] = labels[3]
            else:
                row[1] = labels[4]
        cursor.updateRow(row)

print("✅ Shapefile atualizado com sucesso com a nova coluna 'Jenks'!")


In [ ]:
#buffer do supplier e clip territorial_final
buffer_sup = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\suppliers_bo_buf500"
territorial_final = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Correction\Indigenous_Land\bo_IL_territorial_final_mapa.shp"
#resulta em um novo shape chamada bo_supplier_final


In [ ]:
#fazer o join do territorial final nas provincias
import arcpy

# Caminhos dos arquivos
input_table = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Jurisdiction\Geobolivia\SUNIT\limites_provinciales_alb.shp"
join_table = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Correction\Indigenous_Land\bo_IL_territorial_final.shp"
output_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Correction\Indigenous_Land\bo_IL_territorial_final_mapa.shp"

# Campo de join
join_field = "id_prov"

# Criar uma cópia do input_table para não sobrescrever
arcpy.CopyFeatures_management(input_table, output_shapefile)

# Fazer o join
arcpy.JoinField_management(in_data=output_shapefile,
                           in_field=join_field,
                           join_table=join_table,
                           join_field=join_field)

print("Join realizado com sucesso e shapefile salvo em:")
print(output_shapefile)

In [ ]:
import arcpy

# Caminhos dos shapefiles
output_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Correction\Indigenous_Land\bo_IL_supplier_final.shp"  # Caminho de saída

# Executa o Clip
result = arcpy.Clip_analysis(in_features=territorial_final,
                    clip_features=buffer_sup,
                    out_feature_class=output_features)

print("Clip concluído com sucesso!")

In [ ]:
#calculo risco supplier
#1 calcular a area territorial_final clipada e adicionar a coluna prov_buf
# 2 criar a coluna risk1 que é = linhas de prov_buf * linhas de normalized
#3 somar todas as linhas de risk1 e chegar em uma valor total, criar a coluna SR que é = soma_total_risk1 / area do buffer (=785397.954017)

In [ ]:

field_name = "prov_area"
area_unit = "SQUARE_KILOMETERS"

# Adiciona o campo se ainda não existir
if field_name not in [f.name for f in arcpy.ListFields(output_features)]:
    arcpy.AddField_management(output_features, field_name, "DOUBLE")


In [ ]:
# Calcula a área em km²
arcpy.CalculateGeometryAttributes_management(
    in_features=result,
    geometry_property=[["prov_area", "AREA"]],
    area_unit=area_unit
)

print("Área calculada e salva na coluna 'prov_area'.")

In [ ]:

new_field = "risk1"

# Adiciona o campo se ainda não existir
if new_field not in [f.name for f in arcpy.ListFields(output_features)]:
    arcpy.AddField_management(output_features, new_field, "DOUBLE")

# Calcula o valor do campo como prov_area * normalized
arcpy.CalculateField_management(
    in_table=output_features,
    field=new_field,
    expression="!prov_area! * !Normalized!",
    expression_type="PYTHON3"
)

print("Campo 'risk1' criado e calculado com sucesso.")


In [ ]:

# Nome do novo campo
new_field = "SR"

# Soma todos os valores da coluna 'risk1'
total_risk1 = 0
with arcpy.da.SearchCursor(output_features, ["risk1"]) as cursor:
    for row in cursor:
        total_risk1 += row[0]

# Constante para normalização (ajuste conforme necessário)
constante = 785397.954017
valor_normalizado = total_risk1 / constante

# Adiciona o novo campo se ainda não existir
if new_field not in [f.name for f in arcpy.ListFields(output_features)]:
    arcpy.AddField_management(output_features, new_field, "DOUBLE")

# Preenche todas as linhas com o valor normalizado
with arcpy.da.UpdateCursor(output_features, [new_field]) as cursor:
    for row in cursor:
        row[0] = valor_normalizado
        cursor.updateRow(row)

print(f"Campo '{new_field}' criado com valor {valor_normalizado} em todas as linhas, {total_risk1}")


In [ ]:
import arcpy

# Caminho do shapefile
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Correction\Indigenous_Land\bo_IL_risk_jurisd.shp"

# Nome do campo a ser atualizado
field_name = "Jenks"

# Atualizar os valores nulos ou vazios
with arcpy.da.UpdateCursor(shapefile_path, [field_name]) as cursor:
    for row in cursor:
        if row[0] in (None, "", " "):  # Verifica se está vazio ou nulo
            row[0] = "Very Low"
            cursor.updateRow(row)

print("Valores vazios no campo 'jenks' foram atualizados para 'Very Low'.")

Mexico

In [ ]:
import arcpy

# Caminho do shapefile com os 4 pontos
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\suppliers_mx.shp"

# Nome do novo campo
new_field = "Id_sup"

# Verifica se o campo já existe
existing_fields = [f.name for f in arcpy.ListFields(shapefile_path)]
if new_field not in existing_fields:
    arcpy.AddField_management(shapefile_path, new_field, "TEXT", field_length=10)

# Lista de nomes para os suppliers
supplier_names = ["mx_sup_01", "mx_sup_02", "mx_sup_03", "mx_sup_04"]

# Atualiza os valores no novo campo
with arcpy.da.UpdateCursor(shapefile_path, [new_field]) as cursor:
    for i, row in enumerate(cursor):
        if i < len(supplier_names):
            row[0] = supplier_names[i]
            cursor.updateRow(row)

print("Campo 'Id_sup' criado e valores atribuídos com sucesso.")

In [ ]:
import arcpy
input_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Protected Areas\Protected_Areas_MX_10-25.shp"
output_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Protected Areas\Protected_Areas_MX_10-25_alb.shp"

albers_wkt = """
PROJCS["Conica_Equivalente_de_Albers_Brasil",
GEOGCS["GCS_SIRGAS2000",
DATUM["D_SIRGAS2000",
SPHEROID["Geodetic_Reference_System_of_1980",6378137,298.2572221009113]],
PRIMEM["Greenwich",0],
UNIT["Degree",0.017453292519943295]],
PROJECTION["Albers"],
PARAMETER["standard_parallel_1",-2],
PARAMETER["standard_parallel_2",-22],
PARAMETER["latitude_of_origin",-12],
PARAMETER["central_meridian",-54],
PARAMETER["false_easting",5000000],
PARAMETER["false_northing",10000000],
UNIT["Meter",1]]
"""
spatial_ref = arcpy.SpatialReference()
spatial_ref.loadFromString(albers_wkt)

arcpy.management.Project(
    in_dataset=input_shapefile,
    out_dataset=output_shapefile,
    out_coor_system=spatial_ref
)

print("Shapefile reprojetado com sucesso para Albers SIRGAS 2000!")

In [ ]:
import arcpy
import csv

# Caminhos dos shapefiles
shape1 = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Protected Areas\PA_WDPA_out_2025_classified_mx.shp"
shape2 = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Protected Areas\Mars\CONSOLIDADO\CONSOLIDADO\PA_WDPA_Feb_2025_Municipales_Estatatales_FINAL.shp"

# Colunas a comparar
col_shape1 = "desig_eng"
col_shape2 = "FIRST_DE_1"

# Ler valores únicos das colunas
valores_shape1 = set()
with arcpy.da.SearchCursor(shape1, [col_shape1]) as cursor1:
    for row in cursor1:
        if row[0] is not None:
            valores_shape1.add(row[0].strip())

valores_shape2 = set()
with arcpy.da.SearchCursor(shape2, [col_shape2]) as cursor2:
    for row in cursor2:
        if row[0] is not None:
            valores_shape2.add(row[0].strip())

# Comparar diferenças e interseções
somente_shape1 = sorted(valores_shape1 - valores_shape2)
somente_shape2 = sorted(valores_shape2 - valores_shape1)
comuns = sorted(valores_shape1 & valores_shape2)

# Exportar para CSV
output_csv = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Protected Areas\comparacao_atributos.csv"
with open(output_csv, "w", newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Only_in_Shape1", "Only_in_Shape2", "Common_Values"])

    max_len = max(len(somente_shape1), len(somente_shape2), len(comuns))
    for i in range(max_len):
        row = [
            somente_shape1[i] if i < len(somente_shape1) else "",
            somente_shape2[i] if i < len(somente_shape2) else "",
            comuns[i] if i < len(comuns) else ""
        ]
        writer.writerow(row)

print(f"Arquivo CSV gerado em: {output_csv}")


In [ ]:
import arcpy
import os

# Entradas
municipios_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Jurisdiction\Municipality_mx_itrf.shp"
areas_protegidas_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Protected Areas\PA_WDPA_out_2025_classified_mx.shp"

# Verifica se a pasta de saída existe
output_folder = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Intersect\Protected Area"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Saída
output_fc = os.path.join(output_folder, "mex_PA_mun_inter_teste.shp")

# Executar Intersect
arcpy.Intersect_analysis(
    in_features=[municipios_fc, areas_protegidas_fc],
    out_feature_class=output_fc,
    join_attributes="ALL"
)

print("Intersect concluído. Resultado salvo em:", output_fc)

In [ ]:
import arcpy

# Caminho para o shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Intersect\Protected Area\mex_PA_mun_inter_teste.shp"

# Nome do novo campo
area_field = "area_pa"

# Verifica se o campo já existe
field_names = [f.name for f in arcpy.ListFields(fc)]
if area_field not in field_names:
    arcpy.AddField_management(fc, area_field, "DOUBLE")
    print(f"Campo '{area_field}' criado.")
else:
    print(f"Campo '{area_field}' já existe.")

# Atualiza o campo com a área em hectares
with arcpy.da.UpdateCursor(fc, ["SHAPE@AREA", area_field]) as cursor:
    for row in cursor:
        area_ha = row[0] / 10000  # converte m² para hectares
        row[1] = area_ha
        cursor.updateRow(row)

print("Cálculo de área em hectares concluído e salvo na coluna 'area_pa'.")

In [ ]:
import arcpy
import os

# Entrada
input_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Intersect\Protected Area\mex_PA_mun_inter_teste.shp"

# Saída
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Intersect\Protected Area\mex_PA_mun_diss_test.shp"

# Campos para dissolver
dissolve_fields = ["id_mun"]

# Estatísticas
stats_fields = [
    ["NAME_2", "FIRST"],
    ["wdpaid", "FIRST"],
    ["mun_area", "FIRST"],
    ["area_ha", "FIRST"],
    ["area_pa", "SUM"]
]

# Executar Dissolve
arcpy.Dissolve_management(
    in_features=input_fc,
    out_feature_class=output_fc,
    dissolve_field=dissolve_fields,
    statistics_fields=stats_fields,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES"
)

print("Dissolve concluído. Resultado salvo em:", output_fc)

In [ ]:
import arcpy

# Caminho para o shapefile de entrada
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Intersect\Protected Area\mex_PA_mun_diss_test.shp"

# Caminho para o shapefile de saída final
final_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_PA_territorial_final_test.shp"

# Campos
sum_area_field = "SUM_area_p"
first_area_field = "FIRST_area"
area_prop_field = "Area_prop"
normalized_field = "Normalized"

# Copiar o arquivo original para o novo nome
arcpy.CopyFeatures_management(fc, final_fc)

# Verifica e cria o campo 'Area_prop' se necessário
field_names = [f.name for f in arcpy.ListFields(final_fc)]
if area_prop_field not in field_names:
    arcpy.AddField_management(final_fc, area_prop_field, "DOUBLE")
    print(f"Campo '{area_prop_field}' criado.")
else:
    print(f"Campo '{area_prop_field}' já existe.")

# Preenche o campo 'Area_prop'
with arcpy.da.UpdateCursor(final_fc, [sum_area_field, first_area_field, area_prop_field]) as cursor:
    for row in cursor:
        if row[1] and row[1] != 0:
            row[2] = row[0] / row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("Campo 'Area_prop' preenchido com sucesso.")

# Verifica e cria o campo 'Normalized' se necessário
if normalized_field not in field_names:
    arcpy.AddField_management(final_fc, normalized_field, "DOUBLE")
    print(f"Campo '{normalized_field}' criado.")
else:
    print(f"Campo '{normalized_field}' já existe.")

# Calcula min e max de 'Area_prop'
min_value = None
max_value = None
with arcpy.da.SearchCursor(final_fc, [area_prop_field]) as cursor:
    for row in cursor:
        if min_value is None or row[0] < min_value:
            min_value = row[0]
        if max_value is None or row[0] > max_value:
            max_value = row[0]

print(f"Min Value: {min_value}")
print(f"Max Value: {max_value}")

# Normaliza os valores
with arcpy.da.UpdateCursor(final_fc, [area_prop_field, normalized_field]) as cursor:
    for row in cursor:
        if max_value != min_value:
            normalized_value = (row[0] - min_value) / (max_value - min_value)
        else:
            normalized_value = 0
        row[1] = normalized_value
        cursor.updateRow(row)

print("Normalização concluída com sucesso!")
print("Resultado final salvo em:", final_fc)

In [ ]:
import arcpy

# Caminho do shapefile original
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_PA_territorial_final_test.shp"

# Adiciona uma nova coluna "Jenks" se ela não existir
field_name = "Jenks"
if field_name not in [f.name for f in arcpy.ListFields(shapefile_path)]:
    arcpy.AddField_management(shapefile_path, field_name, "TEXT", field_length=20)

# Define os intervalos e rótulos (bins e labels)
bins = [0.041615, 0.127874, 0.274177, 0.505662]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# Atualiza a coluna "Jenks" com base em "Normalized"
with arcpy.da.UpdateCursor(shapefile_path, ["Normalized", field_name]) as cursor:
    for row in cursor:
        value = row[0]
        if value is None:
            row[1] = None
        else:
            # Verifica em qual intervalo o valor se encaixa
            if value < bins[0]:
                row[1] = labels[0]
            elif value < bins[1]:
                row[1] = labels[1]
            elif value < bins[2]:
                row[1] = labels[2]
            elif value < bins[3]:
                row[1] = labels[3]
            else:
                row[1] = labels[4]
        cursor.updateRow(row)

print("✅ Shapefile atualizado com sucesso com a nova coluna 'Jenks'!")


In [ ]:
#fazer o join do territorial final nas provincias
import arcpy

# Caminhos dos arquivos
input_table = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Jurisdiction\Municipality_mx_itrf.shp"
join_table = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_complete.shp"
output_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_territorial_final_mapa_test.shp"

# Campo de join
join_field = "id_mun"

# Criar uma cópia do input_table para não sobrescrever
arcpy.CopyFeatures_management(input_table, output_shapefile)

# Fazer o join
arcpy.JoinField_management(in_data=output_shapefile,
                           in_field=join_field,
                           join_table=join_table,
                           join_field=join_field)

print("Join realizado com sucesso e shapefile salvo em:")
print(output_shapefile)

In [ ]:
import arcpy
import os

# Caminhos de entrada e saída
input_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_PA_territorial_final_mapa_test.shp"
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\Protected Areas\mx_PA_risk_jurisd_test.shp"

# Copia o shape original para criar o novo
arcpy.CopyFeatures_management(input_fc, output_fc)

# Verifica se o campo 'Jenks' existe
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if "Jenks" not in field_names:
    raise ValueError("O campo 'Jenks' não existe no shapefile de entrada.")

# Atualiza os valores do campo 'Jenks'
with arcpy.da.UpdateCursor(output_fc, ["Jenks"]) as cursor:
    for row in cursor:
        if row[0] in (None, "", " "):
            row[0] = "Very Low"
            cursor.updateRow(row)

print("Shape 2 criado com sucesso e campo 'Jenks' atualizado.")

In [ ]:
import arcpy

# Caminho do shapefile de entrada com os 4 pontos
input_point_supplier = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_itrf.shp"

# Caminho do shapefile de saída com os buffers
buffer_supplier = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_buf250.shp"

# Distância do buffer em metros (250 km = 250000 m)
buffer_distance = "250000 Meters"

# Criar o buffer
arcpy.Buffer_analysis(
    in_features=input_point_supplier,
    out_feature_class=buffer_supplier,
    buffer_distance_or_field=buffer_distance,
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=""
)

print("Buffer de 250 km criado com sucesso!")

# Adicionar campo para área em km²
area_field = "area_km"
field_names = [f.name for f in arcpy.ListFields(buffer_supplier)]
if area_field not in field_names:
    arcpy.AddField_management(buffer_supplier, area_field, "DOUBLE")
    print(f"Campo '{area_field}' criado.")

# Calcular área em km²
with arcpy.da.UpdateCursor(buffer_supplier, ["SHAPE@", area_field]) as cursor:
    for row in cursor:
        area_m2 = row[0].getArea("PLANAR", "SQUAREMETERS")
        area_km2 = area_m2 / 1_000_000  # converter m² para km²
        row[1] = area_km2
        cursor.updateRow(row)

print("Cálculo de área concluído e campo 'area_km' preenchido.")

In [ ]:
#exceção para apenas um supplier
#buffer do supplier e clip territorial_final
buffer_sup = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_buf250.shp"
territorial_final = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_PA_territorial_final_mapa_test.shp"
#resulta em um novo shape chamada bo_supplier_final

import arcpy

# Caminhos dos shapefiles
output_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_PA_supplier_final_test.shp"  # Caminho de saída

# Executa o Clip
result = arcpy.Clip_analysis(in_features=territorial_final,
                    clip_features=buffer_sup,
                    out_feature_class=output_features)

print("Clip concluído com sucesso!")


In [ ]:
import arcpy

# Caminhos dos shapefiles
buffer_sup = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_buf250.shp"
territorial_final = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_PA_territorial_final_mapa_test.shp"
output_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_PA_supplier_final_test.shp"  # Caminho de saída

# Executa o Intersect
arcpy.Intersect_analysis(
    in_features=[territorial_final, buffer_sup],
    out_feature_class=output_features,
    join_attributes="ALL",
    cluster_tolerance="-1 Unknown",
    output_type="INPUT"
)

print("Intersect concluído com sucesso!")

In [ ]:
import arcpy

# Caminho do shapefile de intersect
intersect_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_PA_supplier_final_test.shp"

# Campo de área em km²
area_field = "area_bufm"
field_names = [f.name for f in arcpy.ListFields(intersect_fc)]
if area_field not in field_names:
    arcpy.AddField_management(intersect_fc, area_field, "DOUBLE")
    print(f"Campo '{area_field}' criado.")

# Campo de risco
risk_field = "risk1"
if risk_field not in field_names:
    arcpy.AddField_management(intersect_fc, risk_field, "DOUBLE")
    print(f"Campo '{risk_field}' criado.")

# Calcular área e risco
with arcpy.da.UpdateCursor(intersect_fc, ["SHAPE@", area_field, "Normalized", risk_field]) as cursor:
    for row in cursor:
        area_m2 = row[0].getArea("PLANAR", "SQUAREMETERS")
        area_km2 = area_m2 / 1_000_000
        row[1] = area_km2
        row[3] = area_km2 * row[2] if row[2] is not None else None
        cursor.updateRow(row)

print("Cálculo de área e risco concluído com sucesso!")

In [ ]:
import arcpy

# Caminhos dos shapefiles
input_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_PA_supplier_final_test.shp"
out_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_PA_risk_per_sup_final_test.shp"

# Campo para dissolver
dissolve_fields = ["Id_sup"]

# Campos a manter na tabela de atributos
stats_fields = [
    ["NAME_2", "FIRST"],
    ["mun_area", "FIRST"],
    ["id_mun", "FIRST"],
    ["SUM_area_p", "FIRST"],
    ["Area_prop", "FIRST"],
    ["Normalized", "FIRST"],
    ["Jenks", "FIRST"],
    ["Lat", "FIRST"],
    ["Long", "FIRST"],
    ["risk1", "SUM"],  # Isso será renomeado automaticamente para SUM_risk1
    ["area_km", "FIRST"]
]

# Executar Dissolve
arcpy.Dissolve_management(
    in_features=input_features,
    out_feature_class=out_features,
    dissolve_field=dissolve_fields,
    statistics_fields=stats_fields,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES"
)

print("Dissolve concluído com sucesso!")

# Criar e calcular o campo SR = SUM_risk1 / area_km
sr_field = "SR"
field_names = [f.name for f in arcpy.ListFields(out_features)]
if sr_field not in field_names:
    arcpy.AddField_management(out_features, sr_field, "DOUBLE")
    print(f"Campo '{sr_field}' criado.")

# Calcular SR
with arcpy.da.UpdateCursor(out_features, ["SUM_risk1", "FIRST_ar_1", sr_field]) as cursor:
    for row in cursor:
        if row[1] not in (None, 0):  # evitar divisão por zero
            row[2] = row[0] / row[1]
        else:
            row[2] = None
        cursor.updateRow(row)

print("Campo 'SR' calculado com sucesso!")

In [ ]:
import arcpy

# Caminhos dos shapefiles
polygon_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_PA_risk_per_sup_final.shp"
point_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_itrf.shp"
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_PA_supplier_risk.shp"

# Campos
key_field = "Id_sup"
sr_field = "SR"
jenks_field = "Jenks"

# Intervalos e rótulos
bins = [0.041615, 0.127874, 0.274177, 0.505662]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# 1. Copiar o shape de pontos
arcpy.CopyFeatures_management(point_fc, output_fc)

# 2. Adicionar campo 'SR' se não existir
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if sr_field not in field_names:
    arcpy.AddField_management(output_fc, sr_field, "DOUBLE")

# 3. Adicionar campo 'Jenks' se não existir
if jenks_field not in field_names:
    arcpy.AddField_management(output_fc, jenks_field, "TEXT")

# 4. Criar dicionário Id_sup -> SR
sr_dict = {}
with arcpy.da.SearchCursor(polygon_fc, [key_field, sr_field]) as cursor:
    for row in cursor:
        sr_dict[row[0]] = row[1]

# 5. Atualizar SR e Jenks no novo shape
with arcpy.da.UpdateCursor(output_fc, [key_field, sr_field, jenks_field]) as cursor:
    for row in cursor:
        id_sup = row[0]
        sr_value = sr_dict.get(id_sup, None)
        if sr_value is not None:
            row[1] = sr_value
            # Determinar o rótulo Jenks
            if sr_value < bins[0]:
                row[2] = labels[0]
            elif sr_value < bins[1]:
                row[2] = labels[1]
            elif sr_value < bins[2]:
                row[2] = labels[2]
            elif sr_value < bins[3]:
                row[2] = labels[3]
            else:
                row[2] = labels[4]
            cursor.updateRow(row)

print("Shape atualizado e salvo como 'mx_PA_supplier_risk'.")

In [ ]:
import arcpy

# Caminho para o shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Indigenous Land\mx_ageb_mun_dissolve.shp"

# Campos envolvidos
campo_numerador = "SUM_Spob_t"
campo_denominador = "SUM_Spob_1"
campo_resultado = "Porcen_pop"

# Verifica se o campo já existe, senão cria
field_names = [f.name for f in arcpy.ListFields(fc)]
if campo_resultado not in field_names:
    arcpy.AddField_management(fc, campo_resultado, "DOUBLE")
    print(f"Campo '{campo_resultado}' criado.")
else:
    print(f"Campo '{campo_resultado}' já existe.")

# Calcula Porcen_pop = (SUM_Spob_1 / SUM_Spob_t) * 100
with arcpy.da.UpdateCursor(fc, [campo_numerador, campo_denominador, campo_resultado]) as cursor:
    for row in cursor:
        if row[1] and row[1] != 0:
            row[2] = (row[0] / row[1])
        else:
            row[2] = 0
        cursor.updateRow(row)

print("Cálculo de 'Porcen_pop' em porcentagem concluído com sucesso.")

In [ ]:
import arcpy

# Caminho do shapefile original
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Indigenous Land\mx_ageb_mun_dissolve.shp"

# Caminho do shapefile final
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_il_territorial_final_mapa.shp"

# Copiar para novo shapefile
arcpy.CopyFeatures_management(fc, output_fc)

# Campos
campo_porcen = "Porcen_pop"
campo_jenks = "Jenks"

# Criar campo Jenks se não existir
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if campo_jenks not in field_names:
    arcpy.AddField_management(output_fc, campo_jenks, "TEXT")
    print(f"Campo '{campo_jenks}' criado.")
else:
    print(f"Campo '{campo_jenks}' já existe.")

# Atualizar Jenks com base nos intervalos
with arcpy.da.UpdateCursor(output_fc, [campo_porcen, campo_jenks]) as cursor:
    for row in cursor:
        valor = row[0]
        if valor == 0:
            row[1] = "Not Risk"
        elif valor <= 0.10:
            row[1] = "Very Low"
        elif valor <= 0.20:
            row[1] = "Low"
        elif valor <= 0.40:
            row[1] = "Medium"
        elif valor <= 0.80:
            row[1] = "High"
        else:
            row[1] = "Very High"
        cursor.updateRow(row)

print("Classificação Jenks concluída e shapefile salvo em:", output_fc)

In [ ]:
import arcpy

# Caminhos dos shapefiles
buffer_sup = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_buf250.shp"
territorial_final = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_il_territorial_final_mapa.shp"
output_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_IL_supplier_final.shp"  # Caminho de saída

# Executa o Intersect
arcpy.Intersect_analysis(
    in_features=[territorial_final, buffer_sup],
    out_feature_class=output_features,
    join_attributes="ALL",
    cluster_tolerance="-1 Unknown",
    output_type="INPUT"
)

print("Intersect concluído com sucesso!")

In [ ]:
import arcpy

# Caminho do shapefile
intersect_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_IL_supplier_final.shp"

# Nome dos campos
area_field = "area_bufm"
risk_field = "risk1"

# Verifica e cria os campos se necessário
field_names = [f.name for f in arcpy.ListFields(intersect_fc)]

if area_field not in field_names:
    arcpy.AddField_management(intersect_fc, area_field, "DOUBLE")
    print(f"Campo '{area_field}' criado.")

if risk_field not in field_names:
    arcpy.AddField_management(intersect_fc, risk_field, "DOUBLE")
    print(f"Campo '{risk_field}' criado.")

# 1. Calcular e preencher o campo de área
with arcpy.da.UpdateCursor(intersect_fc, ["SHAPE@", area_field]) as cursor:
    for row in cursor:
        area_m2 = row[0].getArea("PLANAR", "SQUAREMETERS")
        area_km2 = area_m2 / 1_000_000
        row[1] = area_km2
        cursor.updateRow(row)
print("Campo de área preenchido com sucesso.")

# 2. Calcular e preencher o campo de risco
with arcpy.da.UpdateCursor(intersect_fc, [area_field, "Porcen_pop", risk_field]) as cursor:
    for row in cursor:
        row[2] = row[0] * row[1] if row[1] is not None else None
        cursor.updateRow(row)
print("Campo de risco calculado com sucesso.")

In [ ]:
import arcpy

# Caminhos dos shapefiles
input_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_IL_supplier_final.shp"
out_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_IL_risk_per_sup_final.shp"

# Campo para dissolver
dissolve_fields = ["Id_sup"]

# Campos a manter na tabela de atributos
stats_fields = [
    ["SUM_Spob_t", "SUM"],
    ["SUM_Spob_1", "SUM"],
    ["Porcen_pop", "FIRST"],
    ["Id_mun", "FIRST"],
    ["Area_bufm", "SUM"],
    ["area_mun", "SUM"],
    ["Porcen_pop", "FIRST"],
    ["Jenks", "FIRST"],
    ["Lat", "FIRST"],
    ["Long", "FIRST"],
    ["risk1", "SUM"],  # Isso será renomeado automaticamente para SUM_risk1
    ["area_km", "FIRST"]
]

# Executar Dissolve
arcpy.Dissolve_management(
    in_features=input_features,
    out_feature_class=out_features,
    dissolve_field=dissolve_fields,
    statistics_fields=stats_fields,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES"
)

print("Dissolve concluído com sucesso!")

# Criar e calcular o campo SR = SUM_risk1 / area_km
sr_field = "SR"
field_names = [f.name for f in arcpy.ListFields(out_features)]
if sr_field not in field_names:
    arcpy.AddField_management(out_features, sr_field, "DOUBLE")
    print(f"Campo '{sr_field}' criado.")

# Calcular SR
with arcpy.da.UpdateCursor(out_features, ["SUM_risk1", "FIRST_arEA", sr_field]) as cursor:
    for row in cursor:
        if row[1] not in (None, 0):  # evitar divisão por zero
            row[2] = row[0] / row[1]
        else:
            row[2] = None
        cursor.updateRow(row)

print("Campo 'SR' calculado com sucesso!")

In [ ]:
import arcpy

# Caminhos dos shapefiles
polygon_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_IL_risk_per_sup_final.shp"
point_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_itrf.shp"
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_IL_supplier_risk.shp"

# Campos
key_field = "Id_sup"
sr_field = "SR"
jenks_field = "Jenks"

# Intervalos e rótulos
bins = [0.041615, 0.127874, 0.274177, 0.505662]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# 1. Copiar o shape de pontos
arcpy.CopyFeatures_management(point_fc, output_fc)

# 2. Adicionar campo 'SR' se não existir
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if sr_field not in field_names:
    arcpy.AddField_management(output_fc, sr_field, "DOUBLE")

# 3. Adicionar campo 'Jenks' se não existir
if jenks_field not in field_names:
    arcpy.AddField_management(output_fc, jenks_field, "TEXT")

# 4. Criar dicionário Id_sup -> SR
sr_dict = {}
with arcpy.da.SearchCursor(polygon_fc, [key_field, sr_field]) as cursor:
    for row in cursor:
        sr_dict[row[0]] = row[1]

# 5. Atualizar SR e Jenks no novo shape
with arcpy.da.UpdateCursor(output_fc, [key_field, sr_field, jenks_field]) as cursor:
    for row in cursor:
        id_sup = row[0]
        sr_value = sr_dict.get(id_sup, None)
        if sr_value is not None:
            row[1] = sr_value
            # Determinar o rótulo Jenks
            if sr_value < bins[0]:
                row[2] = labels[0]
            elif sr_value < bins[1]:
                row[2] = labels[1]
            elif sr_value < bins[2]:
                row[2] = labels[2]
            elif sr_value < bins[3]:
                row[2] = labels[3]
            else:
                row[2] = labels[4]
            cursor.updateRow(row)

print("Shape atualizado e salvo como 'mx_IL_supplier_risk'.")

In [ ]:
import arcpy

# Caminhos dos shapefiles
polygon_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\mx_IL_risk_per_sup_final.shp"
point_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_itrf.shp"
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\mx_IL_supplier_risk.shp"

# Campos
key_field = "Id_sup"
sr_field = "SR"
jenks_field = "Jenks"

# Novos intervalos (0 a 100)
bins = [10, 20, 40, 80]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# 1. Copiar o shape de pontos
arcpy.CopyFeatures_management(point_fc, output_fc)

# 2. Adicionar campo 'SR' se não existir
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if sr_field not in field_names:
    arcpy.AddField_management(output_fc, sr_field, "DOUBLE")

# 3. Adicionar campo 'Jenks' se não existir
if jenks_field not in field_names:
    arcpy.AddField_management(output_fc, jenks_field, "TEXT")

# 4. Criar dicionário Id_sup -> SR
sr_dict = {}
with arcpy.da.SearchCursor(polygon_fc, [key_field, sr_field]) as cursor:
    for row in cursor:
        sr_dict[row[0]] = row[1]

# 5. Atualizar SR e Jenks no novo shape
with arcpy.da.UpdateCursor(output_fc, [key_field, sr_field, jenks_field]) as cursor:
    for row in cursor:
        id_sup = row[0]
        sr_value = sr_dict.get(id_sup, None)
        if sr_value is not None:
            row[1] = sr_value
            # Classificação com base nos novos bins
            if sr_value <= bins[0]:
                row[2] = labels[0]  # Very Low
            elif sr_value <= bins[1]:
                row[2] = labels[1]  # Low
            elif sr_value <= bins[2]:
                row[2] = labels[2]  # Medium
            elif sr_value <= bins[3]:
                row[2] = labels[3]  # High
            else:
                row[2] = labels[4]  # Very High
            cursor.updateRow(row)

print("Shape atualizado e salvo como 'mx_IL_supplier_risk'.")


In [ ]:
import arcpy

# Caminho para o shapefile de entrada
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\PastDC\mx_def_conv_dissolve.shp"

# Caminho para o shapefile de saída final
final_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_territorial.shp"

# Campos
sum_area_field = "area_dc"
first_area_field = "area_total"
area_prop_field = "Area_prop"
normalized_field = "Normalized"

# Copiar o arquivo original para o novo nome
arcpy.CopyFeatures_management(fc, final_fc)

# Verifica e cria o campo 'Area_prop' se necessário
field_names = [f.name for f in arcpy.ListFields(final_fc)]
if area_prop_field not in field_names:
    arcpy.AddField_management(final_fc, area_prop_field, "DOUBLE")
    print(f"Campo '{area_prop_field}' criado.")
else:
    print(f"Campo '{area_prop_field}' já existe.")

# Preenche o campo 'Area_prop'
with arcpy.da.UpdateCursor(final_fc, [sum_area_field, first_area_field, area_prop_field]) as cursor:
    for row in cursor:
        if row[1] and row[1] != 0:
            row[2] = row[0] / row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("Campo 'Area_prop' preenchido com sucesso.")

# Verifica e cria o campo 'Normalized' se necessário
if normalized_field not in field_names:
    arcpy.AddField_management(final_fc, normalized_field, "DOUBLE")
    print(f"Campo '{normalized_field}' criado.")
else:
    print(f"Campo '{normalized_field}' já existe.")

# Calcula min e max de 'Area_prop'
min_value = None
max_value = None
with arcpy.da.SearchCursor(final_fc, [area_prop_field]) as cursor:
    for row in cursor:
        if min_value is None or row[0] < min_value:
            min_value = row[0]
        if max_value is None or row[0] > max_value:
            max_value = row[0]

print(f"Min Value: {min_value}")
print(f"Max Value: {max_value}")

# Normaliza os valores
with arcpy.da.UpdateCursor(final_fc, [area_prop_field, normalized_field]) as cursor:
    for row in cursor:
        if max_value != min_value:
            normalized_value = (row[0] - min_value) / (max_value - min_value)
        else:
            normalized_value = 0
        row[1] = normalized_value
        cursor.updateRow(row)

print("Normalização concluída com sucesso!")
print("Resultado final salvo em:", final_fc)

In [ ]:
# Accu_sum
import arcpy

# Path to your feature class or shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_territorial.shp"
# Update with your actual path

# Field name where you want to store the accumulated sum
accumulated_field = "Accu_sum"

# Field with the division result
division_field = "Area_prop"

# Sorting the table by the Area_prop field to ensure the accumulation happens in ascending order
# Create a sorted version of the shapefile based on Area_prop field
sorted_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_dissolve_sorted_Area_prop.shp"
arcpy.management.Sort(fc, sorted_fc, [division_field])  # Sorting by the 'Area_prop' field

# Create a cursor to update the rows
total = 0  # This will hold the running total

# Use UpdateCursor to iterate through each row and accumulate the sum
with arcpy.da.UpdateCursor(sorted_fc, [division_field, accumulated_field]) as cursor:
    for row in cursor:
        # Add the division result to the cumulative total
        total += row[0]  # row[0] is the division result field (change if needed)
        row[1] = total  # Set the accumulated sum to the new field
        cursor.updateRow(row)

arcpy.AddMessage("Accumulated sum calculation completed!")


In [ ]:
## Neglig

import arcpy

# Path to your feature class or shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_dissolve_sorted_Area_prop.shp"

# Field names
accumulated_field = "Accu_sum"
category_field = "Neglig"  # New field to store the "Negligible" value

# Add a new field to store the category (if it doesn't already exist)
field_names = [f.name for f in arcpy.ListFields(fc)]
if category_field not in field_names:
    arcpy.AddField_management(fc, category_field, "TEXT", field_length=50)

# Use an UpdateCursor to iterate through each row and apply the logic
with arcpy.da.UpdateCursor(fc, [accumulated_field, category_field]) as cursor:
    for row in cursor:
        # Check if the Accumulated_Sum is less than or equal to 0.05
        if row[0] <= 0.050:
            row[1] = "Negligible"  # Set the value to "Negligible"
        else:
            row[1] = "To_Jenks"  # Leave empty if the condition is not met

        cursor.updateRow(row)  # Update the row with the new value

print("Categorization completed!")


In [ ]:
## Normalized

import arcpy

# Path to your feature class or shapefile
fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_dissolve_sorted_Area_prop.shp"

# Field names
area_field = "Area_prop"  # The field with the area values
normalized_field = "Normalized"  # The new field for normalized values

# Get the minimum and maximum values from the 'Area_prop' field
min_value = None
max_value = None

with arcpy.da.SearchCursor(fc, [area_field]) as cursor:
    for row in cursor:
        if min_value is None or row[0] < min_value:
            min_value = row[0]
        if max_value is None or row[0] > max_value:
            max_value = row[0]

# Print the min and max values for debugging purposes
print(f"Min Value: {min_value}")
print(f"Max Value: {max_value}")

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(fc)]
if normalized_field not in field_names:
    arcpy.AddField_management(fc, normalized_field, "DOUBLE")

# Normalize the 'Area_prop' values and populate the new field
with arcpy.da.UpdateCursor(fc, [area_field, normalized_field]) as cursor:
    for row in cursor:
        # Avoid division by zero in case all values are the same
        if max_value != min_value:
            # Apply normalization formula: (Value - Min) / (Max - Min)
            normalized_value = (row[0] - min_value) / (max_value - min_value)
        else:
            normalized_value = 0  # If all values are the same, set normalized value to 0

        # Update the normalized value in the new field
        row[1] = normalized_value
        cursor.updateRow(row)

print("Normalization completed!")


In [ ]:
import arcpy

# Caminho do shapefile original
shapefile_path = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_dissolve_sorted_Area_prop_to_jenks.shp"

# Adiciona uma nova coluna "Jenks" se ela não existir
field_name = "Jenks"
if field_name not in [f.name for f in arcpy.ListFields(shapefile_path)]:
    arcpy.AddField_management(shapefile_path, field_name, "TEXT", field_length=20)

# Define os intervalos e rótulos (bins e labels)
bins = [0.025408, 0.078211, 0.239234, 0.536325]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# Atualiza a coluna "Jenks" com base em "Normalized"
with arcpy.da.UpdateCursor(shapefile_path, ["Normalized", field_name]) as cursor:
    for row in cursor:
        value = row[0]
        if value is None:
            row[1] = None
        else:
            # Verifica em qual intervalo o valor se encaixa
            if value < bins[0]:
                row[1] = labels[0]
            elif value < bins[1]:
                row[1] = labels[1]
            elif value < bins[2]:
                row[1] = labels[2]
            elif value < bins[3]:
                row[1] = labels[3]
            else:
                row[1] = labels[4]
        cursor.updateRow(row)

print("✅ Shapefile atualizado com sucesso com a nova coluna 'Jenks'!")


In [ ]:
#fazer o join do territorial final nas provincias
import arcpy

# Caminhos dos arquivos
input_table = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Jurisdiction\Municipality_mx_itrf.shp"
join_table = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_complete.shp"
output_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_territorial_final_mapa.shp"

# Campo de join
join_field = "id_mun"

# Criar uma cópia do input_table para não sobrescrever
arcpy.CopyFeatures_management(input_table, output_shapefile)

# Fazer o join
arcpy.JoinField_management(in_data=output_shapefile,
                           in_field=join_field,
                           join_table=join_table,
                           join_field=join_field)

print("Join realizado com sucesso e shapefile salvo em:")
print(output_shapefile)

In [ ]:
import arcpy
import os

# Caminhos de entrada e saída
input_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_territorial_final_mapa.shp"
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_risk_jurisd.shp"

# Copia o shape original para criar o novo
arcpy.CopyFeatures_management(input_fc, output_fc)

# Verifica se o campo 'Jenks' existe
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if "Jenks" not in field_names:
    raise ValueError("O campo 'Jenks' não existe no shapefile de entrada.")

# Atualiza os valores do campo 'Jenks'
with arcpy.da.UpdateCursor(output_fc, ["Jenks"]) as cursor:
    for row in cursor:
        if row[0] in (None, "", " "):
            row[0] = "Negligible"
            cursor.updateRow(row)

print("Shape 2 criado com sucesso e campo 'Jenks' atualizado.")

In [ ]:
import arcpy

# Caminhos dos shapefiles
buffer_sup = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_buf250.shp"
territorial_final = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_risk_jurisd.shp"
output_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_supplier_final.shp"  # Caminho de saída

# Executa o Intersect
arcpy.Intersect_analysis(
    in_features=[territorial_final, buffer_sup],
    out_feature_class=output_features,
    join_attributes="ALL",
    cluster_tolerance="-1 Unknown",
    output_type="INPUT"
)

print("Intersect concluído com sucesso!")

In [ ]:
import arcpy

# Caminho do shapefile de intersect
intersect_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_supplier_final.shp"

# Campo de área em km²
area_field = "area_bufm"
field_names = [f.name for f in arcpy.ListFields(intersect_fc)]
if area_field not in field_names:
    arcpy.AddField_management(intersect_fc, area_field, "DOUBLE")
    print(f"Campo '{area_field}' criado.")

# Campo de risco
risk_field = "risk1"
if risk_field not in field_names:
    arcpy.AddField_management(intersect_fc, risk_field, "DOUBLE")
    print(f"Campo '{risk_field}' criado.")

# Calcular área e risco
with arcpy.da.UpdateCursor(intersect_fc, ["SHAPE@", area_field, "Normalized", risk_field]) as cursor:
    for row in cursor:
        area_m2 = row[0].getArea("PLANAR", "SQUAREMETERS")
        area_km2 = area_m2 / 1_000_000
        row[1] = area_km2
        row[3] = area_km2 * row[2] if row[2] is not None else None
        cursor.updateRow(row)

print("Cálculo de área e risco concluído com sucesso!")

In [ ]:
import arcpy

# Caminhos dos shapefiles
input_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_supplier_final.shp"
out_features = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_risk_per_sup_final.shp"

# Campo para dissolver
dissolve_fields = ["Id_sup"]

# Campos a manter na tabela de atributos
stats_fields = [
    ["Id_mun", "FIRST"],
    ["Area_bufm", "SUM"],
    ["Jenks", "FIRST"],
    ["Lat", "FIRST"],
    ["Long", "FIRST"],
    ["risk1", "SUM"],
    ["area_km", "FIRST"]
]

# Executar Dissolve
arcpy.Dissolve_management(
    in_features=input_features,
    out_feature_class=out_features,
    dissolve_field=dissolve_fields,
    statistics_fields=stats_fields,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES"
)

print("Dissolve concluído com sucesso!")

# Criar e calcular o campo SR = SUM_risk1 / area_km
sr_field = "SR"
field_names = [f.name for f in arcpy.ListFields(out_features)]
if sr_field not in field_names:
    arcpy.AddField_management(out_features, sr_field, "DOUBLE")
    print(f"Campo '{sr_field}' criado.")

# Calcular SR
with arcpy.da.UpdateCursor(out_features, ["SUM_risk1", "FIRST_arEA", sr_field]) as cursor:
    for row in cursor:
        if row[1] not in (None, 0):  # evitar divisão por zero
            row[2] = row[0] / row[1]
        else:
            row[2] = None
        cursor.updateRow(row)

print("Campo 'SR' calculado com sucesso!")

In [ ]:
import arcpy

# Caminhos dos shapefiles
polygon_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_risk_per_sup_final.shp"
point_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Supplier\suppliers_mx_itrf.shp"
output_fc = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Results\PastDC\mx_pastdc_supplier_risk.shp"

# Campos
key_field = "Id_sup"
sr_field = "SR"
jenks_field = "Jenks"

# Intervalos e rótulos
bins = [0.025408, 0.078211, 0.239234, 0.536325]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

# 1. Copiar o shape de pontos
arcpy.CopyFeatures_management(point_fc, output_fc)

# 2. Adicionar campo 'SR' se não existir
field_names = [f.name for f in arcpy.ListFields(output_fc)]
if sr_field not in field_names:
    arcpy.AddField_management(output_fc, sr_field, "DOUBLE")

# 3. Adicionar campo 'Jenks' se não existir
if jenks_field not in field_names:
    arcpy.AddField_management(output_fc, jenks_field, "TEXT")

# 4. Criar dicionário Id_sup -> SR
sr_dict = {}
with arcpy.da.SearchCursor(polygon_fc, [key_field, sr_field]) as cursor:
    for row in cursor:
        sr_dict[row[0]] = row[1]

# 5. Atualizar SR e Jenks no novo shape
with arcpy.da.UpdateCursor(output_fc, [key_field, sr_field, jenks_field]) as cursor:
    for row in cursor:
        id_sup = row[0]
        sr_value = sr_dict.get(id_sup, None)
        if sr_value is not None:
            row[1] = sr_value
            # Determinar o rótulo Jenks
            if sr_value < bins[0]:
                row[2] = labels[0]
            elif sr_value < bins[1]:
                row[2] = labels[1]
            elif sr_value < bins[2]:
                row[2] = labels[2]
            elif sr_value < bins[3]:
                row[2] = labels[3]
            else:
                row[2] = labels[4]
            cursor.updateRow(row)

print("Shape atualizado e salvo como 'mx_PA_supplier_risk'.")

mexico parte 2

In [ ]:
import arcpy

# Caminhos para os arquivos de entrada
grasslands_2020 = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Diana\Grasslands_2020-Mexico\grassland_mx_2020.shp"
grasslands_2024 = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Diana\Grassland 2024\mx_grasslands_2024.shp"

# Caminho para o GeoPackage de saída
output_gpkg = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\PastDC\grassland_20-24_identity.gpkg"
output_layer_name = "grasslands_identity"

# Caminho completo para salvar dentro do GPKG
output_path = f"{output_gpkg}|layername={output_layer_name}"



In [ ]:
# Executa a operação Identity
arcpy.analysis.Identity(
    in_features=grasslands_2020,
    identity_features=grasslands_2024,
    out_feature_class=output_path
)

print("Operação Identity concluída com sucesso!")

In [ ]:
import arcpy
from arcpy.sa import *

# Ativar extensão Spatial Analyst
arcpy.CheckOutExtension("Spatial")

# Caminhos dos rasters de entrada
raster_2020 = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Diana\Grasslands_2020-Mexico\grassland_mx_2020.tif"
raster_2024 = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Diana\Grassland 2024\grassland_mx_2024.tif"

# Carregar os rasters
r2020 = Raster(raster_2020)
r2024 = Raster(raster_2024)

# Identificar áreas que eram classe 2 ou 3 em 2020 e viraram classe 1 em 2024
changed_areas = Con(((r2020 == 2) | (r2020 == 3)) & (r2024 == 1), 1, 0)

# Salvar o raster de resultado
output_raster = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Diana\Grasslands_2020-Mexico\changed_2or3_to_1.tif"
changed_areas.save(output_raster)

# Converter o raster para shapefile
output_shapefile = r"C:\Users\LíviaCarolineCésarDi\Documents\McDs\Mexico\Diana\Grasslands_2020-Mexico\changed_2or3_to_1.shp"
arcpy.RasterToPolygon_conversion(
    in_raster=output_raster,
    out_polygon_features=output_shapefile,
    simplify="NO_SIMPLIFY",
    raster_field="Value"
)

print("✅ Processamento concluído. Raster e shapefile salvos com sucesso.")
